In [ ]:
import pandas as pd
import requests
import zipfile
import io
import os
import csv
import ctypes as ct
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import re
from functools import reduce


- "https://www.bankofengland.co.uk/-/media/boe/files/markets/asset-purchase-facility/gilt-purchase-operational-results.xlsx"
- "https://www.bankofengland.co.uk/-/media/boe/files/markets/asset-purchase-facility/gilt-sales-time-series.xlsx"

In [ ]:
df_buy = pd.ExcelFile("downloads/gilt-purchase-operational-results.XLSX")

In [ ]:
df_buy.sheet_names

In [ ]:
df_buy = pd.read_excel("downloads/gilt-purchase-operational-results.XLSX", sheet_name="APF Gilts", header=1)

In [ ]:
df_buy.columns

In [ ]:
df_boe = df_buy[['Operation date','Total allocation (proceeds £mn)', 'Total allocation (nominal £mn)']].copy()

In [ ]:
df_sell = pd.ExcelFile("downloads/gilt-sales-time-series.XLSX")

In [ ]:
df_sell.sheet_names

In [ ]:
df_sell = pd.read_excel("downloads/gilt-sales-time-series.XLSX", sheet_name="APF gilt sales", header=1)

In [ ]:
df_sell.columns

In [ ]:
df_sell['Operation date'] = pd.to_datetime(df_sell['Operation date'], format='%d-%m-%Y')
last_date = df_sell['Operation date'].min()

In [ ]:
from bgs.load_gilt_details import load_csv_blocks
from bgs.load_bgs_prices import load_prices
from bgs.load_bgs_amounts import load_bgs_amounts
from bgs.bgs_utils import clean_date, clean_percentage
from bgs.gilt_analytics import coupon_schedule


In [ ]:
details = load_csv_blocks("downloads/BGSDetails.csv")

conv_details = details['Conventionals']
conv_details['%'] = conv_details['%'].apply(clean_percentage)
conv_details['Sequence'] = conv_details['Sequence'].apply(lambda x: np.int64(x))

new_details = details['Index-Linked New-style']
new_details['%'] = new_details['%'].apply(clean_percentage)
new_details['Sequence'] = new_details['Sequence'].apply(lambda x: np.int64(x))

old_details = details['Index-Linked Old-style']
old_details['%'] = old_details['%'].apply(clean_percentage)
old_details['Sequence'] = old_details['Sequence'].apply(lambda x: np.int64(x))

date_variables = ["Latest redemption date","Issue date","First coupon payable on date",]
for date_var in date_variables:
    conv_details[date_var] = conv_details[date_var].apply(clean_date)
    old_details[date_var] = old_details[date_var].apply(clean_date)
    new_details[date_var] = new_details[date_var].apply(clean_date)

price_df=load_prices("downloads/BGSPrices.csv")
price_df.index = pd.to_datetime(price_df.index, format="%d %b %Y").to_period('M').to_timestamp('M')
for text in ['Amalgamated', 'Redeemed', 'redeemed']:
    price_df = price_df.replace(text, 0)
price_df = price_df.replace('missing', None)
price_df = price_df.ffill(axis=0)
price_df = price_df.fillna(0)
price_df.columns = [np.int64(x) for x in price_df.columns]

In [ ]:
gilt_level_buys = df_buy[[
    'ISIN',
    'Operation date',
    'Total allocation (proceeds £mn)',
    'Total allocation (nominal £mn)'
    ]].copy()
gilt_level_buys.set_index('Operation date', inplace=True)
gilt_level_buys.index = pd.to_datetime(gilt_level_buys.index, format='%d-%m-%Y').to_period('M').to_timestamp('M')
gilt_level_buys.reset_index(inplace=True)

In [ ]:
gilt_buys_total = gilt_level_buys.reset_index().groupby([ 'ISIN']).agg(
    Total_Allocation_Proceeds=('Total allocation (proceeds £mn)', 'sum'),
    Total_Allocation_Nominal=('Total allocation (nominal £mn)', 'sum'),
    Last_Purchase_Month=('Operation date', 'max')
)
gilt_buys_total.reset_index(inplace=True)
gilt_buys_total.head()

In [ ]:
gilt_buys_total['maturity_date'] = gilt_buys_total['ISIN'].map(
    lambda x: conv_details[
        conv_details['ISIN Code']==x
        ]['Latest redemption date'].values[0]
    )
gilt_buys_total['bgs_id'] = gilt_buys_total['ISIN'].map(
    lambda x: conv_details[
        conv_details['ISIN Code']==x
        ]['Sequence'].values[0]
    )

In [ ]:
gilt_buys_total.head()

In [ ]:
gilt_buys_total['maturity_date'] = pd.to_datetime(
    gilt_buys_total['maturity_date'], format="%d %b %Y"
)

In [ ]:
gilt_level_sales = df_sell[[
    'ISIN',
    'Operation date',
    'Total allocation (proceeds £mn)',
    'Total allocation (nominal £mn)'
    ]].copy()
gilt_level_sales.set_index('Operation date', inplace=True)
gilt_level_sales.index = pd.to_datetime(gilt_level_sales.index, format='%d-%m-%Y').to_period('M').to_timestamp('M')
gilt_level_sales.reset_index(inplace=True)
gilt_level_sales.head()

In [ ]:
gilt_sales_total = gilt_level_sales.reset_index().groupby([ 'ISIN']).agg(
    Total_sales_proceeds=('Total allocation (proceeds £mn)', 'sum'),
    Total_nominal_reduction=('Total allocation (nominal £mn)', 'sum'),
)
gilt_sales_total.reset_index(inplace=True)

In [ ]:
boe_portfolio = gilt_buys_total.set_index('ISIN').merge(
    gilt_sales_total[[
        'ISIN',
        'Total_nominal_reduction',
        'Total_sales_proceeds'
    ]].set_index('ISIN'),
        how='left',
        left_index=True,
        right_index=True,
        suffixes=('_buy', '_sell')
        ).copy().fillna(0).reset_index()

In [ ]:
boe_portfolio.head()

In [ ]:
boe_portfolio['Coupon']=boe_portfolio['ISIN'].map(
    lambda x: conv_details[
        conv_details['ISIN Code']==x
        ]['%'].values[0]
    )

In [ ]:
today = "2025-08-31"
boe_current = boe_portfolio.loc[boe_portfolio['maturity_date']>today].copy()

In [ ]:
boe_current.head()

In [ ]:
avg_coupon = ((boe_current['Total_Allocation_Nominal'] - boe_current['Total_nominal_reduction'])*boe_current['Coupon']).sum()/(boe_current['Total_Allocation_Nominal'] - boe_current['Total_nominal_reduction']).sum()
apf_funding = (boe_current['Total_Allocation_Proceeds'] - boe_current['Total_sales_proceeds']).sum()
base_rate = 4
(base_rate - avg_coupon) * apf_funding / 100

In [ ]:
ons_download_pusf = "downloads/MF7A.csv"
df_pusf = pd.read_csv(ons_download_pusf, skiprows=8,names=["CDID","MF7A"])
q_start =  df_pusf.loc[df_pusf['CDID']=="1997 APR"].index[0]
df_mf7a_m = df_pusf.iloc[q_start:].ffill()
df_mf7a_m['date']=pd.to_datetime(df_mf7a_m['CDID'], format="%Y %b")
df_mf7a_m.set_index('date', inplace=True)
# df_mf7a_m = df_mf7a_m.resample('ME').ffill()
df_mf7a_m.tail(10)
# df_mf7a_m['MF7A'] = df_mf7a_m['MF7A'].apply(lambda x: x/100)

In [ ]:
df_mf7a_m['MF7A'].sum()

In [ ]:
(boe_current['Total_Allocation_Proceeds']*boe_current['Total_nominal_reduction']/boe_current['Total_Allocation_Nominal'] - boe_current['Total_sales_proceeds']).sum()

In [ ]:
price_df.index.max()

In [ ]:
price_df.loc['2025-04-30', 32100]

In [ ]:
boe_current['mkt_price'] = boe_current['bgs_id'].map(
    lambda x: price_df.loc['2025-04-30', int(x)])

In [ ]:
((boe_current['Total_Allocation_Proceeds'] - boe_current['Total_sales_proceeds']) - (boe_current['Total_Allocation_Nominal'] - boe_current['Total_nominal_reduction'])*boe_current['mkt_price']/100).sum()

In [ ]:
isin_list = ['GB00BK5CVX03', 'GB00BTHH2R79', 'GB00BL68HJ26', 'GB00BYZW3G56', 'GB00BNNGP668', 'GB00BDRHNP05', 'GB00BMBL1G81', 'GB00BFX0ZL78', 'GB00BLPK7227', 'GB00BJMHB534', 'GB00BL68HH02', 'GB00BMGR2809', 'GB00BM8Z2T38', 'GB00BM8Z2S21', 'GB00BMGR2916', 'GB00BZB26Y51', 'GB00BLPK7334', 'GB00BJQWYH73', 'GB00B84Z9V04', 'GB00BNNGP775', 'GB00BDCHBW80', 'GB00BFWFPP71', 'GB00BMBL1F74', 'GB00BLH38158', 'GB00BM8Z2V59', 'GB00BJLR0J16', 'GB00BD0XH204', 'GB00BMBL1D50', 'GB00BFMCN652', 'GB00BLBDX619']

In [ ]:
boe_lme = boe_current.loc[boe_current['ISIN'].isin(isin_list)]

In [ ]:
((boe_lme['Total_Allocation_Proceeds'] - boe_lme['Total_sales_proceeds']) - (boe_lme['Total_Allocation_Nominal'] - boe_lme['Total_nominal_reduction'])*boe_lme['mkt_price']/100).sum()

In [ ]:
boe_current['lme_flag'] = boe_current['ISIN'].isin(isin_list)

In [ ]:
(((boe_current['Total_Allocation_Proceeds'] - boe_current['Total_sales_proceeds']) - (boe_current['Total_Allocation_Nominal'] - boe_current['Total_nominal_reduction'])*boe_current['mkt_price']/100)*boe_current['lme_flag']).sum()

In [ ]:
boe_current.head()

In [ ]:
boe_curves = "downloads/GLC Nominal month end data_2025 to present.xlsx"
df_curves = pd.ExcelFile(boe_curves)
df_curve = pd.read_excel(boe_curves, sheet_name='4. spot curve', skiprows=3)
df_curve = df_curve.set_index('years:')
spot_curve = df_curve.loc['2025-04-30'].to_dict()


In [ ]:
flat_long = spot_curve.get(40)
boe_current['lme_coupon'] = ((boe_current['maturity_date'].dt.year-2025) + (boe_current['maturity_date'].dt.month)//6*0.5).apply(
    lambda x: spot_curve.get(x, flat_long))
boe_current['new_coupon'] = boe_current.apply(lambda x: x['lme_coupon'] if x['lme_flag']==1 else x['Coupon'], axis=1)

In [ ]:
new_avg_coupon = ((boe_current['Total_Allocation_Nominal'] - boe_current['Total_nominal_reduction'])*boe_current['new_coupon']).sum()/(boe_current['Total_Allocation_Nominal'] - boe_current['Total_nominal_reduction']).sum()
(base_rate - new_avg_coupon) * apf_funding / 100